In [1]:
import pandas as pd
import nltk
import re

In [2]:
df = pd.read_csv('../datasets/bullying_twitter_preprocessing.csv')
df.columns=['original_text', 'target']

#### Retirar usernames, links e hashtags

In [3]:
# Regex for username, links, and hashtags
patterns = [
    (r"\@\w+", "__USERNAME__"),
    (r"\bhttps?:(\/\/)?\w+\.\w+(\.\w+)*(\/\w+)*\/?", "__HTTP_LINK__"),
    (r"#", "")
]

In [4]:
for p, s in patterns:
    df.original_text = df.original_text.apply(lambda t: re.sub(p, s, t.lower()))

#### Quebrar em unigrams, bigrams e pos tag

In [5]:
df['unigrams'] = df.original_text.apply(lambda t: nltk.word_tokenize(t))
df['bigrams'] = df.unigrams.apply(lambda t: list(nltk.bigrams(t)))
df['pos_tag'] = df.unigrams.apply(lambda u: list(nltk.pos_tag(u)))

In [7]:
df.to_csv('../datasets/bullying_twitter_preprocessed.csv', index=False, sep=';')

In [9]:
df.original_text = df.original_text.apply(lambda t: t.replace('bullying', ''))

## Conversão para features

In [12]:
# creating the feature matrix 
from sklearn.feature_extraction.text import CountVectorizer
unigram_features = pd.DataFrame(CountVectorizer().fit_transform(df['original_text']).toarray())
unigram_features['target'] = df.target
unigram_features.to_csv('../datasets/bullying_twitter_unigrams_sem_bullying.csv', index=False)

In [13]:
bigram_features = pd.DataFrame(CountVectorizer(ngram_range=(1,2)).fit_transform(df['original_text']).toarray())
bigram_features['target'] = df.target
bigram_features.to_csv('../datasets/bullying_twitter_bigrams_sem_bullying.csv', index=False)